출처: https://wikidocs.net/262585

## LLM 에 도구 바인딩(Binding Tools)

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("agent-bind-tools")

## LLM에 바인딩할 Tool 정의

- get_word_length : 단어의 길이를 반환하는 함수
- add_function : 두 숫자를 더하는 함수
- naver_news_crawl : 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구를 정의합니다.
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]


In [ ]:
from langchain_openai import ChatOpenAI

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)


In [ ]:
llm_with_tools.invoke("What is the length of the word 'teddynote'?").tool_calls

# [{
#     'name': 'get_word_length',  
#     'args': {'word': 'teddynote'},  
#     'id': 'call_ReeUlCuSd35Marox3yaxMgcH',  
#     'type': 'tool_call'
# }]

In [ ]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")


In [ ]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

'''
[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]

==========

get_word_length
{'word': 'teddynote'}
'''



In [ ]:
tool_call_results[0]["type"], tools[0].name
# ('get_word_length', 'get_word_length')


In [ ]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행합니다.
execute_tool_calls(tool_call_results)


# [실행도구] get_word_length
# [실행결과] 9

## bind_tools + Parser + Execution

In [ ]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls


In [ ]:
chain.invoke("What is the length of the word 'teddynote'?")
# [실행도구] get_word_length
# [실행결과] 9

# 실행 결과
chain.invoke("114.5 + 121.2")
print(114.5 + 121.2)
# [실행도구] add_function
# [실행결과] 235.7
# 235.7

chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)
# [실행도구] naver_news_crawl
# [실행결과] [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표
# ...

## bind_tools > Agent & AgentExecutor 로 대체

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length, add_function, naver_news_crawl]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)


In [ ]:
# Agent 실행
result = agent_executor.invoke({"input": "How many letters in the word `teddynote`?"})

# 결과 확인
print(result["output"])

# The word "teddynote" has 9 letters.

In [ ]:
# Agent 실행
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])

#114.5 + 121.2의 계산 결과는 235.7입니다.

In [ ]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

# 결과 확인
print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)

# [1m> Entering new AgentExecutor chain...[0m
# [32;1m[1;3m
# Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


# [0m[33;1m[1;3m235.7[0m[32;1m[1;3m
# Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


# [0m[33;1m[1;3m144.3[0m[32;1m[1;3m
# Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


# [0m[33;1m[1;3m380.0[0m[32;1m[1;3m114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.[0m

# [1m> Finished chain.[0m
# 114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.
# ==========

# 380.0

In [ ]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result["output"])